# Train small Splicevo model

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import tqdm
import os

## Simulate small dataset

To train a Splicevo model, we need the following data:

- Input DNA sequences of shape `B, L, 4` (one-hot encoded)
- Labels of shape `B, L` corresponding to splice site annotations (0=not a splice site, 1=splice donor, 2=splice acceptor)
- SSE values of shape `B, L, N` correspondint to splice site strength estimates in N different conditions (tissues, developmental time points)
- Species vector of length `B` indicating the species for each sequence in the batch
- Mappings of species indices to species names, and condition indices to condition names

`B` is the batch size, `L` is the sequence length (consisting of core sequence `core_len` and `context_len` on both sides), and `N` is the number of conditions.

In [ ]:
# Simulate small dataset
B = 256
L = 2900
core_len = 2000
context_len = 450
N = 5

# Simulate random sequence input
sequences = np.random.randint(0, 2, size=(B, L, 4)).astype(np.float32)

# Simulate species assignment
species = np.random.randint(0, 3, size = B).astype(np.int32)

# Insert splice sites into random sequences
def insert_splice_sites(sequences, donor_freq=0.1, acceptor_freq=0.1):
    B = sequences.shape[0]
    L = sequences.shape[1]
    labels = np.zeros((B, L), dtype=np.int64)
    for b in range(sequences.shape[0]):
        donor_sites = np.random.randint(2, L, int(donor_freq * L))
        acceptor_sites = np.random.randint(2, L, int(acceptor_freq * L))
        for pos in donor_sites:
            labels[b, pos] = 1
            sequences[b, pos-2:pos, :] = [[0, 0, 1, 0],[0, 0, 0, 1]]  # 'GT' donor site
        for pos in acceptor_sites:
            labels[b, pos] = 2
            sequences[b, pos-2:pos, :] = [[1, 0, 0, 0],[0, 0, 1, 0]]  # 'AG' acceptor site
    return sequences, labels

sequences, labels = insert_splice_sites(sequences)

# Print number of sites per species
for i in np.unique(species):
    labels_i = labels[species == i]
    print(f"Species {i}: {labels_i.shape[0]} sequences" )
    for site_type in range(3):
        print(f"  site type {site_type}: {labels_i[labels_i == site_type].shape[0]}")

In [ ]:
# Simulate SSE values distribution 
sse = np.random.beta(2, 0.5, size=(B, L, N)).astype(np.float32)
# Set non-splice positions to zero SSE
sse[labels == 0] = 0
# Set a fraction of splice positions to zero SSE
splice_positions = np.where(labels > 0)
num_zero_sse = int(0.3 * len(splice_positions[0]))
zero_sse_indices = np.random.choice(len(splice_positions[0]), num_zero_sse, replace=False)
sse[splice_positions[0][zero_sse_indices], splice_positions[1][zero_sse_indices], np.random.randint(0, N, size=1)] = 0

# Plot SSE distribution
import matplotlib.pyplot as plt
plt.hist(sse[labels > 0].flatten(), bins=50)
plt.xlabel('SSE value')
plt.ylabel('Frequency')
plt.show()

Split dataset into train (80%) and test (20%)

In [ ]:
# Train/test/validation split
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Split into train+val and test (80/20)
train_val_idx, test_idx = train_test_split(
    np.arange(len(sequences)),
    test_size=0.2,
    random_state=42
)

# Split train+val into train and validation (80/20 of train+val = 64/16 overall)
train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=0.2,
    random_state=42
)

print(f"Train set size: {len(train_idx)}")
print(f"Validation set size: {len(val_idx)}")
print(f"Test set size: {len(test_idx)}")

# Create train, validation, and test sets
train_sequences, train_labels, train_sse, train_species = (
    sequences[train_idx], labels[train_idx], sse[train_idx], species[train_idx]
)
val_sequences, val_labels, val_sse, val_species = (
    sequences[val_idx], labels[val_idx], sse[val_idx], species[val_idx]
)
test_sequences, test_labels, test_sse, test_species = (
    sequences[test_idx], labels[test_idx], sse[test_idx], species[test_idx]
)

# Create datasets
from splicevo.training import SpliceDataset

train_dataset = SpliceDataset(train_sequences, train_labels, train_sse, train_species)
val_dataset = SpliceDataset(val_sequences, val_labels, val_sse, val_species)
test_dataset = SpliceDataset(test_sequences, test_labels, test_sse, test_species)

# Create data loaders
batch_size = 16
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print("\nData loaders created successfully!")

In [ ]:
# Count splice sites in each set
def count_splice_sites(labels):
    donor_count = np.sum(labels == 1)
    acceptor_count = np.sum(labels == 2)
    return donor_count, acceptor_count
train_donors, train_acceptors = count_splice_sites(train_labels)
val_donors, val_acceptors = count_splice_sites(val_labels)
test_donors, test_acceptors = count_splice_sites(test_labels)
print(f"Train set: {train_donors} donors, {train_acceptors} acceptors")
print(f"Validation set: {val_donors} donors, {val_acceptors} acceptors")
print(f"Test set: {test_donors} donors, {test_acceptors} acceptors")

In [ ]:
# Plot SSE distribution for splice sites in each set
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.hist(train_sse[train_labels > 0].flatten(), bins=50)
plt.title('Train Set SSE Distribution')
plt.xlabel('SSE value')
plt.ylabel('Frequency')
plt.subplot(1, 3, 2)
plt.hist(val_sse[val_labels > 0].flatten(), bins=50)
plt.title('Validation Set SSE Distribution')
plt.xlabel('SSE value')
plt.ylabel('Frequency')
plt.subplot(1, 3, 3)
plt.hist(test_sse[test_labels > 0].flatten(), bins=50)
plt.title('Test Set SSE Distribution')
plt.xlabel('SSE value')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

## Train Splicevo model

In [ ]:
from splicevo.model import SplicevoModel
from splicevo.training import SpliceTrainer, SpliceDataset
import torch

# Model configuration
model_config = {
    'embed_dim': 128,
    'num_resblocks': 16,
    'dilation_strategy': 'alternating',
    'alternate': 4,
    'num_classes': 3,
    'n_conditions': N,
    'context_len': context_len,
    'num_heads': 8,
    'dropout': 0.3,
    'usage_loss_type': 'weighted_mse',
    'n_species': len(np.unique(species))
}

# Initialize model
model = SplicevoModel(**model_config)
print("Model initialized:")
print(f"  Encoder: Conv blocks with multi-scale fusion")
print(f"  Transformer: {model_config['num_heads']} attention heads")
print(f"  Output heads: Splice classification + SSE prediction")

In [ ]:
# Training configuration
training_config = {
    'learning_rate': 1.0e-3,
    'weight_decay': 1.0e-3,
    'num_epochs': 100,
    'splice_weight': 0.5,
    'usage_weight': 0.5,
    'weighted_mse_extreme_low': 0.1,
    'weighted_mse_extreme_high': 0.9,
    'weighted_mse_extreme_weight': 5.0,
    'early_stopping_patience': 10
}

# Create trainer
trainer = SpliceTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    learning_rate=training_config['learning_rate'],
    weight_decay=training_config['weight_decay'],
    splice_weight=training_config['splice_weight'],
    usage_weight=training_config['usage_weight'],
    weighted_mse_extreme_low=training_config['weighted_mse_extreme_low'],
    weighted_mse_extreme_high=training_config['weighted_mse_extreme_high'],
    weighted_mse_extreme_weight=training_config['weighted_mse_extreme_weight'],
    usage_loss_type=model_config['usage_loss_type'],
)

print("Trainer created successfully!")

In [ ]:
# Train model
print("Training model...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

history = trainer.train(
    n_epochs=training_config['num_epochs'],
    early_stopping_patience=training_config['early_stopping_patience']
)

print("Training complete!")


# Predict with the model

In [ ]:
# Set model in evaluation mode
model.eval()

Calculate predictions on the test set 

In [ ]:
all_splice_preds = []
all_splice_true = []
all_usage_preds = []
all_usage_true = []

with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
        sequences = batch['sequences'].to(device)
        splice_labels = batch['splice_labels'].to(device)
        usage_targets = batch['usage_targets'].to(device)
        species_ids = batch['species_ids'].to(device) if 'species_ids' in batch else None
        
        # Forward pass
        outputs = model(sequences, species_ids=species_ids)
        
        # Get predictions
        splice_logits = outputs['splice_logits']  # (batch, central_len, num_classes)
        splice_preds = torch.argmax(splice_logits, dim=-1)  # (batch, central_len)
        
        usage_preds = torch.sigmoid(outputs['usage_predictions'])  # (batch, central_len, n_conditions)
        
        # Move to CPU and convert to numpy
        all_splice_preds.append(splice_preds.cpu().numpy())
        all_splice_true.append(splice_labels.cpu().numpy())
        all_usage_preds.append(usage_preds.cpu().numpy())
        all_usage_true.append(usage_targets.cpu().numpy())
        
        if (batch_idx + 1) % 10 == 0:
            print(f"  Processed {batch_idx + 1}/{len(test_loader)} batches")

# Concatenate all predictions
splice_preds_all = np.concatenate(all_splice_preds)  # (n_test, central_len)
splice_true_all = np.concatenate(all_splice_true)    # (n_test, central_len)
usage_preds_all = np.concatenate(all_usage_preds)    # (n_test, central_len, n_conditions)
usage_true_all = np.concatenate(all_usage_true)      # (n_test, central_len, n_conditions)

print(f"\nPredictions collected:")
print(f"  Splice predictions shape: {splice_preds_all.shape}")
print(f"  Splice ground truth shape: {splice_true_all.shape}")
print(f"  Usage predictions shape: {usage_preds_all.shape}")
print(f"  Usage ground truth shape: {usage_true_all.shape}")

Compute evaluation metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt

# Flatten for overall accuracy
splice_preds_flat = splice_preds_all.flatten()
splice_true_flat = splice_true_all.flatten()

accuracy = accuracy_score(splice_true_flat, splice_preds_flat)
print(f"\nSplice site classification:")
print(f"  Overall accuracy: {accuracy:.4f}")

# Per-class metrics
precision, recall, f1, support = precision_recall_fscore_support(
    splice_true_flat, splice_preds_flat, 
    labels=[0, 1, 2],
    zero_division=0
)
class_names = ['Not splice', 'Donor', 'Acceptor']
for i, name in enumerate(class_names):
    print(f"  {name:15} - Precision: {precision[i]:.4f}, Recall: {recall[i]:.4f}, F1: {f1[i]:.4f}")

# Usage prediction metrics (MSE)
usage_mse = np.mean((usage_preds_all - usage_true_all) ** 2)
print(f"\nUsage prediction:")
print(f"  Mean squared error: {usage_mse:.6f}")

# Per-condition MSE
for cond_idx in range(usage_preds_all.shape[-1]):
    cond_mse = np.mean((usage_preds_all[..., cond_idx] - usage_true_all[..., cond_idx]) ** 2)
    print(f"  Condition {cond_idx} MSE: {cond_mse:.6f}")

In [ ]:
np.unique(splice_true_flat)

In [ ]:
np.unique(splice_preds_flat)